In [ ]:
import numpy as np
import pandas as pd
import ydata_profiling
from ydata_profiling import ProfileReport
import sqlite3
import tqdm

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
path = 'emisivos.csv'
df = pd.read_csv(path, sep=';')
df.tail()

,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,...,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef,CoefTot,latitud,longitud
20597,3,NaN,1,Aereo,2023-02-04,15740,2023-02-21,15757,1,Uruguay,...,4420.00,281.38,122.00,0.00,1728.35,0.0,230.8,461.61,-33.527135,-58.212337
20598,3,Aeropuerto de Carrasco,1,Aereo,2023-02-10,15746,2023-02-17,15753,1,Uruguay,...,2719.49,93.98,344.74,0.00,5132.32,0.0,230.8,923.22,-34.863757,-56.216238
20599,3,NaN,1,Aereo,2023-01-25,15730,2023-02-06,15742,1,Uruguay,...,7770.88,986.22,649.09,128.65,2804.63,0.0,230.8,923.22,-34.863989,-56.176821
20600,3,NaN,1,Aereo,2023-02-13,15749,2023-02-25,15761,1,Uruguay,...,6656.00,332.11,0.00,187.80,3456.70,0.0,230.8,923.22,-34.375276,-55.224414
20601,3,Aeropuerto de Carrasco,1,Aereo,2023-02-04,15740,2023-02-24,15760,1,Uruguay,...,16882.65,3672.13,978.42,61.70,6203.83,0.0,230.8,923.22,-34.851086,-56.045893


In [ ]:
df.duplicated().sum()

0

In [ ]:
#df[['IdLugarSalida', 'Lugar Salida']]
df['Lugar Salida'].unique()

array(['Chuy', nan, 'Aeropuerto de Carrasco', 'Río Branco', 'Fray Bentos',
       'Bella Unión', 'Colonia', 'Salto', 'Paysandú', 'Carmelo',
       'Melo - Aceguá', 'Artigas', 'Rivera', 'Puerto de montevideo',
       'Nueva Palmira', 'Otros'], dtype=object)

In [ ]:
df[(df['IdLugarSalida'] == 18) & (df['Lugar Salida'] != 'Río Branco')].count()

IdLugarSalida                          51
Lugar Salida                            0
IdTranspSalidad                        51
Transporte Internacional de Salida     51
FechaSalida                            51
IdFecSalida                            51
FechaEntrada                           51
IdFecEntrada                           51
IdNacionalidad                         51
Pais                                   51
IdDeptoResidencia                      50
Departamento                           50
IdMotivo                               49
Motivo                                 49
IdOcupacion                            51
Ocupacion                              51
IdNivelEstudio                         51
Estudio                                51
IdDestino                              51
Destino                                51
IdAlojamiento                          51
Alojamiento                            51
IdLugarIngreso                         51
Lugar Ingreso                     

In [ ]:
import re

def check_date_format(string):
    pattern = r"\d{4}-\d{2}-\d{2}"
    match = re.fullmatch(pattern, string)
    if match:
        return True
    else:
        return False

# Testing the function
fechas_bien = sum([check_date_format(fecha) for fecha in df['FechaEntrada']])
fechas_bien

20602

In [ ]:
from datetime import datetime

date_format = "%Y-%m-%d"
fecha_entrada = np.array([datetime.strptime(date, date_format) for date in df['FechaEntrada']])

date_format_salida = "%m/%d/%Y"

fechas_salida_mal = list(df['FechaSalida'])
fecha_salida = []
for fecha in fechas_salida_mal:
    try:
        fecha_bien = datetime.strptime(fecha, date_format_salida)
        fecha_bien.strftime(date_format)
    except:
        fecha_bien = datetime.strptime(fecha, date_format)

    fecha_salida.append(fecha_bien)

fecha_salida = np.array(fecha_salida)

estadia = np.array(df['Estadia'])

comp = np.array([(fecha_entrada[i] - fecha_salida[i]).days == estadia[i] for i in range(len(estadia))])
diff = np.array([(fecha_entrada[i] - fecha_salida[i]).days for i in range(len(estadia))])

count_true = np.count_nonzero(comp == True)
count_false = np.count_nonzero(comp == False)
print (count_true)
print (count_false)

df_dates = pd.DataFrame({'FechaEntrada': fecha_entrada, 'FechaSalida': fecha_salida, 'Fechas_dif': diff, 'estadia': estadia})
conn = sqlite3.connect ('emisivos.db')
df_dates.to_sql(name= 'fechas', con=conn, if_exists= 'replace')

20391
211


20602

In [ ]:
display(df[df.duplicated(keep = 'first')].head())
df[df.duplicated(keep = 'first')].shape

,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,...,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef,CoefTot,latitud,longitud


(0, 43)

In [ ]:
display(df.nunique())

IdLugarSalida                             15
Lugar Salida                              15
IdTranspSalidad                            6
Transporte Internacional de Salida         6
FechaSalida                             2920
IdFecSalida                             1621
FechaEntrada                            1418
IdFecEntrada                            1418
IdNacionalidad                            20
Pais                                      20
IdDeptoResidencia                         19
Departamento                              19
IdMotivo                                  10
Motivo                                    10
IdOcupacion                               21
Ocupacion                                 21
IdNivelEstudio                             8
Estudio                                    8
IdDestino                                 13
Destino                                   13
IdAlojamiento                             13
Alojamiento                               13
IdLugarIng

In [ ]:
subset_columns = list(df.columns)
subset_columns.pop(subset_columns.index('FechaSalida')) # Se saca esta por tener distintos formatos para un mismo ID
subset_columns.pop(subset_columns.index('Lugar Salida')) # Se saca esta por tener distintos valores para un mismo ID

df[df.duplicated(subset=subset_columns, keep=False)].shape

(0, 43)

In [ ]:
gastos_columns = ['GastoAlojamiento', 'GastoAlimentacion', 'GastoCompras', 'GastoCultural', 'GastoResto', 'GastoTours', 'GastoTransporteInternac', 'GatoTransporteLocal', 'GastoTotal']
df_d_gastos = df[df.duplicated(subset=gastos_columns,keep=False)].sort_values(by = gastos_columns)
df_d_gastos_nnz = df_d_gastos[(df_d_gastos[gastos_columns] > 0).any(axis=1)].sort_values(by = gastos_columns)

c = df.columns
display(df_d_gastos_nnz[c[0:20]][174:180])
display(df_d_gastos_nnz[c[20:40]][174:180])
df_d_gastos_nnz[c[40:43]][174:180]
#df_d_gastos_nnz.shape


,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,IdDeptoResidencia,Departamento,IdMotivo,Motivo,IdOcupacion,Ocupacion,IdNivelEstudio,Estudio,IdDestino,Destino
2685,2,Puerto de montevideo,5,Maritimo - Fluvial,2017-05-26,13660,2017-05-29,13663,1,Uruguay,21.0,Montevideo,2.0,Visita familiares / amigos,16,"Empl. Adm, Cajero, Vendedor",4,Secundaria completa,10,Argentina
2686,7,Fray Bentos,3,Terrestre Bus,2017-05-01,13635,2017-05-05,13639,1,Uruguay,54.0,Treinta y Tres,5.0,Tratamiento Salud,16,"Empl. Adm, Cajero, Vendedor",4,Secundaria completa,10,Argentina
13320,8,Paysandú,2,Terrestre Auto,2019-08-06,14462,2019-08-08,14464,1,Uruguay,61.0,Paysandu,2.0,Visita familiares / amigos,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,10,Argentina
13321,8,Paysandú,2,Terrestre Auto,08/06/2019,14462,2019-08-08,14464,10,Argentina,61.0,Paysandu,2.0,Visita familiares / amigos,16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,10,Argentina
2120,7,Fray Bentos,3,Terrestre Bus,2017-04-13,13617,2017-04-17,13621,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",6,Terciaria completa,10,Argentina
2121,7,NaN,3,Terrestre Bus,04/14/2017,13618,2017-04-17,13621,1,Uruguay,13.0,Soriano,2.0,Visita familiares / amigos,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,10,Argentina


,IdAlojamiento,Alojamiento,IdLugarIngreso,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,IdTranspLocal,Trasporte Local,Estadia,Gente,GastoTotal,GastoAlojamiento,GastoAlimentacion,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef
2685,6,Vivienda Familiares / Amigos,2,Puerto de montevideo,5,Maritimo - Fluvial,0,Sin Datos,3,2,190.96,0.0,66.54,0.0,0.00,0.0,0.0,0.0,124.41,161.49
2686,6,Vivienda Familiares / Amigos,7,Fray Bentos,3,Terrestre Bus,0,Sin Datos,4,1,190.96,0.0,66.54,0.0,0.00,0.0,0.0,0.0,124.41,81.96
13320,6,Vivienda Familiares / Amigos,8,Paysandú,2,Terrestre Auto,1,Auto propio,2,2,300.00,0.0,67.44,0.0,108.79,0.0,0.0,0.0,123.77,211.47
13321,6,Vivienda Familiares / Amigos,8,Paysandú,2,Terrestre Auto,1,Auto propio,2,2,300.00,0.0,67.44,0.0,108.79,0.0,0.0,0.0,123.77,107.31
2120,6,Vivienda Familiares / Amigos,7,Fray Bentos,3,Terrestre Bus,0,Sin Datos,4,2,200.00,0.0,68.57,0.0,31.43,0.0,0.0,0.0,100.00,107.10
2121,6,Vivienda Familiares / Amigos,7,Fray Bentos,3,Terrestre Bus,0,Sin Datos,3,2,200.00,0.0,68.57,0.0,31.43,0.0,0.0,0.0,100.00,107.10


,CoefTot,latitud,longitud
2685,322.98,-34.818097,-56.137911
2686,81.96,-33.233454,-54.390465
13320,422.95,-32.360346,-57.201348
13321,214.62,-31.932027,-57.889241
2120,214.21,-34.917569,-56.152041
2121,214.21,-33.864404,-57.371208


In [ ]:
gastos_columns = ['GastoAlojamiento', 'GastoAlimentacion', 'GastoCompras', 'GastoCultural', 'GastoResto', 'GastoTours', 'GastoTransporteInternac', 'GatoTransporteLocal', 'GastoTotal']
subset_columns = list(df.columns)
subset_columns.pop(subset_columns.index('Coef'))
subset_columns.pop(subset_columns.index('CoefTot'))
subset_columns.pop(subset_columns.index('latitud'))
subset_columns.pop(subset_columns.index('longitud'))
subset_columns.pop(subset_columns.index('FechaSalida')) # Se saca esta por tener distintos formatos para un mismo ID
subset_columns.pop(subset_columns.index('Lugar Salida')) # Se saca esta por tener distintos valores para un mismo ID

df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)
df_duplicates_nnz = df_duplicates[(df_duplicates[gastos_columns] > 0).any(axis=1)]

print (df_duplicates.shape)
print (df_duplicates_nnz.shape)

c = df.columns
display(df_duplicates[c[0:20]][175:181])
display(df_duplicates[c[20:40]][175:181])
df_duplicates[c[40:43]][175:181]

conn = sqlite3.connect ('emisivos.db')
df_dates.to_sql(name= 'duplicados', con=conn, if_exists= 'replace')

(245, 43)
(241, 43)


,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,IdDeptoResidencia,Departamento,IdMotivo,Motivo,IdOcupacion,Ocupacion,IdNivelEstudio,Estudio,IdDestino,Destino
6365,7,Fray Bentos,3,Terrestre Bus,2017-11-21,13839,2017-11-27,13845,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,10,Argentina
6366,7,Fray Bentos,3,Terrestre Bus,2017-11-21,13839,2017-11-27,13845,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,10,Argentina
6358,7,Fray Bentos,3,Terrestre Bus,11/22/2017,13840,2017-11-27,13845,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,10,Argentina
6362,7,Fray Bentos,3,Terrestre Bus,2017-11-22,13840,2017-11-27,13845,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,10,Argentina
18809,7,Fray Bentos,3,Terrestre Bus,2022-11-30,15674,2022-12-03,15677,10,Argentina,41.0,Flores,9.0,Estudios,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,10,Argentina
19391,7,Fray Bentos,3,Terrestre Bus,2022-11-30,15674,2022-12-03,15677,10,Argentina,41.0,Flores,9.0,Estudios,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,10,Argentina


,IdAlojamiento,Alojamiento,IdLugarIngreso,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,IdTranspLocal,Trasporte Local,Estadia,Gente,GastoTotal,GastoAlojamiento,GastoAlimentacion,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef
6365,1,Hotel,7,Fray Bentos,3,Terrestre Bus,3,Taxi - Bus,6,2,1510.14,521.09,285.71,35.13,125.50,0.0,0.0,188.32,389.52,65.32
6366,1,Hotel,7,Fray Bentos,3,Terrestre Bus,3,Taxi - Bus,6,2,1510.14,521.09,285.71,35.13,125.50,0.0,0.0,188.32,389.52,65.32
6358,1,Hotel,7,Fray Bentos,3,Terrestre Bus,3,Taxi - Bus,5,2,1289.83,434.24,238.09,35.13,104.58,0.0,0.0,188.32,324.60,65.32
6362,1,Hotel,7,Fray Bentos,3,Terrestre Bus,3,Taxi - Bus,5,2,1289.83,434.24,238.09,35.13,104.58,0.0,0.0,188.32,324.60,65.32
18809,6,Vivienda Familiares / Amigos,1,Colonia,5,Maritimo - Fluvial,3,Taxi - Bus,3,1,50.00,0.00,20.00,120.00,30.00,0.0,0.0,0.00,0.00,535.66
19391,6,Vivienda Familiares / Amigos,1,Colonia,5,Maritimo - Fluvial,3,Taxi - Bus,3,1,50.00,0.00,20.00,120.00,30.00,0.0,0.0,0.00,0.00,79.35


20602

In [ ]:
cols_rep = ['Coef', 'CoefTot', 'latitud', 'longitud']
df_duplicates_cols_rep = df_duplicates[cols_rep]

print((df_duplicates_cols_rep == 'Sin Datos').any(axis=1).sum())
print((df_duplicates_cols_rep.isnull()).any(axis=1).sum())
(df_duplicates_cols_rep == 'Desconocido / Sin Datos').any(axis=1).sum()
#null_count = column.isnull().sum()
#sin_datos_count = (column == "Sin Datos").sum()
#desc_sin_datos_count = (column == "Desconocido / Sin Datos").sum()

0
0


0

In [ ]:
def duplicados(df):
    subset_columns = list(df.columns)
    cols_not_to_consider = ['Coef', 'CoefTot', 'latitud', 'longitud', 'FechaSalida', 'Lugar Salida']

    # Genero listado de columnas a partir de las cuales se buscan duplicados
    for col in cols_not_to_consider:
        subset_columns.pop(subset_columns.index(col))

    # Genero dataframe con duplicados
    df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)

    dup_cont = [] # listado con índices de filas de duplicados contradictorios
    dup_no_cont = [] # listado con índices de filas de duplicados NO contradictorios

    for row in df_duplicates.index:
        df_dup_row = df_duplicates[(df_duplicates[subset_columns] == df.loc[row][subset_columns]).all(axis=1)] # Dataframe con duplicados para cada registro duplicado

        # Genero listas con los atributos que definiran si el duplicado es contradictorio o no
        for col in cols_not_to_consider:
            coef_list = [coef for coef in df_dup_row['Coef']]
            coef_tot_list = [coef_tot for coef_tot in df_dup_row['CoefTot']]
            lat_list = [lat for lat in df_dup_row['latitud']]
            lon_list = [lon for lon in df_dup_row['longitud']]
        check_list = [coef_list, coef_tot_list, lat_list, lon_list]

        to_remove = [None, 'Sin Datos', 'Desconocido / Sin Datos'] # valores a remover para chequear si los restantes son contradictorios

        # Chequeo si los duplicados son contradictorios
        for lista in check_list:
            contradiccion = []
            for value in to_remove:
                lista.remove(value) if value in lista else lista
            if len(lista) == len(set(lista)):
                contradiccion.append(True) # Existen valores distintos luego de sacar los nulos o faltantes
            else:
                contradiccion.append(False) # No existen valores distintos luego de sacar los nulos o faltantes

        dup_cont.append(row) if True in contradiccion else dup_no_cont.append(row)

    return {'ratio_cont' : round(len(dup_cont)/len(df),4),'ratio_no_cont': round(len(dup_no_cont)/len(df),4)}

duplicados(df)

NameError: name 'df' is not defined

In [ ]:
def duplicados_row (df,row):
    subset_columns = list(df.columns)
    cols_not_to_consider = ['Coef', 'CoefTot', 'latitud', 'longitud', 'FechaSalida', 'Lugar Salida']

    # Genero listado de columnas a partir de las cuales se buscan duplicados
    for col in cols_not_to_consider:
        subset_columns.pop(subset_columns.index(col))

    # Genero dataframe con duplicados
    df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)

    df_dup_row = df_duplicates[(df_duplicates[subset_columns] == df.loc[row][subset_columns]).all(axis=1)] # Dataframe con duplicados para cada registro duplicado

    if len(df_dup_row) == 0:
        no_duplicacion = 1
        no_contradiccion = 1
        return {'no_duplicacion': no_duplicacion, 'no_contradiccion': no_contradiccion}

    # Genero listas con los atributos que definiran si el duplicado es contradictorio o no
    for col in cols_not_to_consider:
        coef_list = [coef for coef in df_dup_row['Coef']]
        coef_tot_list = [coef_tot for coef_tot in df_dup_row['CoefTot']]
        lat_list = [lat for lat in df_dup_row['latitud']]
        lon_list = [lon for lon in df_dup_row['longitud']]
    check_list = [coef_list, coef_tot_list, lat_list, lon_list]

    to_remove = [None, 'Sin Datos', 'Desconocido / Sin Datos'] # valores a remover para chequear si los restantes son contradictorios

    # Chequeo si los duplicados son contradictorios
    for lista in check_list:
        contradiccion = []
        for value in to_remove:
            lista.remove(value) if value in lista else lista
        if len(lista) == len(set(lista)):
            contradiccion.append(True) # Existen valores distintos luego de sacar los nulos o faltantes
        else:
            contradiccion.append(False) # No existen valores distintos luego de sacar los nulos o faltantes

    if True in contradiccion:
         no_duplicacion = 1
         no_contradiccion = 0
    else:
        no_duplicacion = 0
        no_contradiccion = 1

    return {'no_duplicacion': no_duplicacion, 'no_contradiccion': no_contradiccion}



In [ ]:
for col in df.columns:
    print (f'emisivos.{col}')

emisivos.IdLugarSalida
emisivos.Lugar Salida
emisivos.IdTranspSalidad
emisivos.Transporte Internacional de Salida
emisivos.FechaSalida
emisivos.IdFecSalida
emisivos.FechaEntrada
emisivos.IdFecEntrada
emisivos.IdNacionalidad
emisivos.Pais
emisivos.IdDeptoResidencia
emisivos.Departamento
emisivos.IdMotivo
emisivos.Motivo
emisivos.IdOcupacion
emisivos.Ocupacion
emisivos.IdNivelEstudio
emisivos.Estudio
emisivos.IdDestino
emisivos.Destino
emisivos.IdAlojamiento
emisivos.Alojamiento
emisivos.IdLugarIngreso
emisivos.Lugar Ingreso
emisivos.IdTranspIngreso
emisivos.Transporte Internacional de Ingreso
emisivos.IdTranspLocal
emisivos.Trasporte Local
emisivos.Estadia
emisivos.Gente
emisivos.GastoTotal
emisivos.GastoAlojamiento
emisivos.GastoAlimentacion
emisivos.GastoTransporteInternac
emisivos.GatoTransporteLocal
emisivos.GastoCultural
emisivos.GastoTours
emisivos.GastoCompras
emisivos.GastoResto
emisivos.Coef
emisivos.CoefTot
emisivos.latitud
emisivos.longitud


In [ ]:
dups = []
for row in df_duplicates.index:
    dups.append(duplicados_row(df, row))

dups

[{'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 1},
 {'no_duplicacion': 1, 'no_contradiccion': 1},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplicacion': 1, 'no_contradiccion': 0},
 {'no_duplica

In [ ]:
df.duplicated()

0    False
1    False
2    False
3    False
dtype: bool

In [ ]:
df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)
df_duplicates

,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,...,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef,CoefTot,latitud,longitud
1168,1,Colonia,5,Maritimo - Fluvial,02/23/2017,13568,2017-03-01,13574,1,Uruguay,...,70.0,144.0,0.00,0.0,268.00,588.0,150.92,301.84,-34.760733,-56.010821
1169,1,Colonia,5,Maritimo - Fluvial,2017-02-23,13568,2017-03-01,13574,1,Uruguay,...,70.0,144.0,0.00,0.0,268.00,588.0,150.92,301.84,-34.378340,-55.955990
1782,1,Colonia,5,Maritimo - Fluvial,2017-03-28,13601,2017-04-03,13607,1,Uruguay,...,64.0,0.0,0.00,26.0,174.00,0.0,194.34,388.67,-34.896953,-56.188011
1783,1,Colonia,5,Maritimo - Fluvial,2017-03-28,13601,2017-04-03,13607,1,Uruguay,...,64.0,0.0,0.00,26.0,174.00,0.0,194.34,388.67,-34.856935,-56.137360
1755,1,Colonia,5,Maritimo - Fluvial,2017-04-06,13610,2017-04-10,13614,1,Uruguay,...,64.0,88.0,200.00,0.0,174.00,0.0,194.34,388.67,-34.894935,-56.291948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19953,13,Rivera,2,Terrestre Auto,01/02/2023,15707,2023-01-12,15717,1,Uruguay,...,0.0,250.0,91.64,0.0,77.66,30.7,91.58,102.29,-32.330425,-58.070908
19869,13,Rivera,2,Terrestre Auto,2023-01-09,15714,2023-01-12,15717,1,Uruguay,...,0.0,200.0,0.00,0.0,0.00,200.0,91.58,102.29,-30.901576,-55.542132
19870,13,Rivera,2,Terrestre Auto,2023-01-09,15714,2023-01-12,15717,1,Uruguay,...,0.0,200.0,0.00,0.0,0.00,200.0,91.58,102.29,-30.897817,-55.556356
1172,18,Río Branco,2,Terrestre Auto,2017-03-12,13585,2017-03-20,13593,1,Uruguay,...,0.0,320.0,348.00,0.0,196.00,0.0,100.29,401.15,-34.908845,-56.150340


In [ ]:
df_duplicates_cols_rep[df_duplicates_cols_rep.duplicated()]

,Coef,CoefTot,latitud,longitud


In [ ]:
import re

def check_date_format(string):
    pattern = r"\d{2}/\d{2}/\d{4}"
    match = re.fullmatch(pattern, string)
    if match:
        return True
    else:
        return False


In [ ]:
import re

def check_two_decimals(value):
    if isinstance(value, (int, float)):
        decimals = str(value).split(".")[1:]
        if len(decimals) == 1 and len(decimals[0]) == 2:
            return True
    return False


check_two_decimals(50.1200)

True

In [ ]:
import re

def check_no_special_characters(input_string):
    pattern = r'^[a-zA-Z0-9]+$'
    match = re.match(pattern, str(input_string))
    return match is not None



False

In [ ]:
def check_precision(number):
    return True if (np.round(number,2) == number and np.round(number,1) != number and len(str(number)) > 5) else False

check_precision(45.28)

False

In [ ]:
str(format(45.1545648, ".2f")).split('.')

['45', '15']

In [ ]:
'ho.la'.split('.')

['ho', 'la']

In [ ]:
c = df.columns
display(df[c[0:20]].head())
display(df[c[20:40]].head())
df[c[40:43]].head()

,IdLugarSalida,Lugar Salida,IdTranspSalidad,Transporte Internacional de Salida,FechaSalida,IdFecSalida,FechaEntrada,IdFecEntrada,IdNacionalidad,Pais,IdDeptoResidencia,Departamento,IdMotivo,Motivo,IdOcupacion,Ocupacion,IdNivelEstudio,Estudio,IdDestino,Destino
0,4,Chuy,2,Terrestre Auto,2016-12-29,13512,2017-01-10,13524,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",15,"Obrero esp, Conductor, Artesano",4,Secundaria completa,20,Brasil
1,18,NaN,2,Terrestre Auto,2016-12-29,13512,2017-01-09,13523,1,Uruguay,31.0,Maldonado,NaN,NaN,3,Estudiante,5,Terciaria incompleta,20,Brasil
2,3,Aeropuerto de Carrasco,1,Aereo,2017-01-03,13517,2017-01-27,13541,1,Uruguay,20.0,Canelones,1.0,"Ocio, Recreo, Vacaciones",11,"Patron, Com, Ind, Prod Agrop",4,Secundaria completa,40,Europa
3,3,Aeropuerto de Carrasco,1,Aereo,2017-01-06,13520,2017-01-21,13535,1,Uruguay,21.0,Montevideo,1.0,"Ocio, Recreo, Vacaciones",16,"Empl. Adm, Cajero, Vendedor",5,Terciaria incompleta,20,Brasil
4,4,Chuy,2,Terrestre Auto,2017-01-02,13516,2017-01-25,13539,1,Uruguay,20.0,Canelones,1.0,"Ocio, Recreo, Vacaciones",11,"Patron, Com, Ind, Prod Agrop",6,Terciaria completa,20,Brasil


,IdAlojamiento,Alojamiento,IdLugarIngreso,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,IdTranspLocal,Trasporte Local,Estadia,Gente,GastoTotal,GastoAlojamiento,GastoAlimentacion,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef
0,5,Vivienda Arrendada,4,Chuy,2,Terrestre Auto,0,Sin Datos,12,5,1507.12,981.39,350.50,0.0,175.24,0.00,0.00,0.00,0.00,89.61
1,1,Hotel,18,Río Branco,2,Terrestre Auto,0,Sin Datos,11,5,3275.00,1100.00,880.00,0.0,385.00,300.00,235.00,210.00,165.00,96.18
2,6,Vivienda Familiares / Amigos,3,Aeropuerto de Carrasco,1,Aereo,0,Sin Datos,24,2,4785.75,0.00,1129.75,214.0,836.85,523.03,256.29,2039.83,0.00,129.04
3,2,Appart Hotel,3,Aeropuerto de Carrasco,1,Aereo,0,Sin Datos,15,3,5883.00,2228.41,1782.73,117.0,779.94,445.68,0.00,311.98,334.26,129.04
4,5,Vivienda Arrendada,4,Chuy,2,Terrestre Auto,0,Sin Datos,23,5,3400.00,1800.00,933.12,0.0,408.24,152.14,0.00,106.50,0.00,89.61


,CoefTot,latitud,longitud
0,448.05,-34.871881,-56.256221
1,480.88,-34.913901,-54.952648
2,258.09,-34.867642,-56.052078
3,387.13,-34.878356,-56.108557
4,448.05,-34.827786,-56.008889


In [ ]:
dict_coord = {
    'Montevideo': {'max_lat': -34.7833, 'min_lat': -34.9300, 'max_lon': -56.0126, 'min_lon': -56.4000},
    'Maldonado': {'max_lat': -34.5666, 'min_lat': -34.9700, 'max_lon': -54.9127, 'min_lon': -55.4084},
    'Canelones': {'max_lat': -34.5340, 'min_lat': -34.9572, 'max_lon': -55.4167, 'min_lon': -56.4526},
    'Salto': {'max_lat': -31.2005, 'min_lat': -31.4503, 'max_lon': -57.6760, 'min_lon': -57.9723},
    'Lavalleja': {'max_lat': -33.1833, 'min_lat': -34.6729, 'max_lon': -54.7500, 'min_lon': -55.3333},
    'San José': {'max_lat': -33.0000, 'min_lat': -34.6311, 'max_lon': -56.1385, 'min_lon': -56.7500},
    'Río Negro': {'max_lat': -32.5000, 'min_lat': -33.8902, 'max_lon': -56.1667, 'min_lon': -58.0667},
    'Durazno': {'max_lat': -32.0675, 'min_lat': -33.6650, 'max_lon': -55.5169, 'min_lon': -57.4614},
    'Cerro Largo': {'max_lat': -31.9333, 'min_lat': -33.7691, 'max_lon': -53.9167, 'min_lon': -55.4750},
    'Colonia': {'max_lat': -33.1334, 'min_lat': -34.6827, 'max_lon': -56.1864, 'min_lon': -58.5143},
    'Tacuarembó': {'max_lat': -31.0833, 'min_lat': -32.6508, 'max_lon': -54.6000, 'min_lon': -56.6000},
    'Treinta y Tres': {'max_lat': -32.5333, 'min_lat': -34.4231, 'max_lon': -53.7500, 'min_lon': -55.3333},
    'Rocha': {'max_lat': -33.1000, 'min_lat': -34.7524, 'max_lon': -53.5833, 'min_lon': -54.7500},
    'Artigas': {'max_lat': -29.5833, 'min_lat': -30.7685, 'max_lon': -56.0000, 'min_lon': -57.8000},
    'Paysandú': {'max_lat': -31.1167, 'min_lat': -32.5227, 'max_lon': -57.0000, 'min_lon': -58.1000},
    'Rivera': {'max_lat': -30.8333, 'min_lat': -31.9759, 'max_lon': -55.4000, 'min_lon': -56.5000},
    'Flores': {'max_lat': -32.5167, 'min_lat': -33.6493, 'max_lon': -55.0833, 'min_lon': -56.3333},
    'Soriano': {'max_lat': -32.6333, 'min_lat': -33.8216, 'max_lon': -57.4167, 'min_lon': -58.5833},
    'Florida': {'max_lat': -33.1000, 'min_lat': -34.1117, 'max_lon': -55.5833, 'min_lon': -57.3333},
}

def dist_lat_norm (latitud, departamento):
    max_lat = dict_coord[departamento]['max_lat']
    min_lat = dict_coord[departamento]['min_lat']
    if latitud > max_lat:
        distancia_norm = abs(latitud - max_lat) / abs(max_lat - (90))
    elif latitud < min_lat:
        distancia_norm = abs(latitud - min_lat) / abs(min_lat - (-90))
    else:
        distancia_norm = 0

    return round(distancia_norm,4)


def dist_lon_norm (longitud, departamento):
    max_lon = dict_coord[departamento]['max_lon']
    min_lon = dict_coord[departamento]['min_lon']
    if longitud > max_lon:
        distancia_norm = abs(longitud - max_lon) / abs(max_lon - (180))
    elif longitud < min_lon:
        distancia_norm = abs(longitud - min_lon) / abs(min_lon - (-180))
    else:
        distancia_norm = 0

    return round(distancia_norm,4)

def validate_lat(lat):
    return 1 if lat > -90 and lat < 90 else 0

print (validate_lat(25))
print (validate_lat(100))
print (validate_lat(-100))
print (validate_lat(-45))
print (validate_lat(0))



print (dist_lon_norm(30,'Montevideo'))
print (dist_lon_norm(0,'Montevideo'))
print (dist_lon_norm(-56.3,'Montevideo'))
print (dist_lon_norm(-60,'Montevideo'))
print (dist_lon_norm(-90*2,'Montevideo'))




1
0
0
1
1
0.3644
0.2373
0
0.0291
1.0


In [ ]:
#print(df['Lugar Salida'].unique())
#set(df['Lugar Salida'].unique()) == set(df['Lugar Ingreso'].unique())
lugsal = set(df['Lugar Salida'].unique())
lugsal.remove('Otros')
lugsal.remove(np.nan)

lugin = set(df['Lugar Ingreso'].unique())
#lugin.remove('Otros')
#lugin.remove(np.nan)

#set(df['Lugar Ingreso'].unique())
lugin == lugsal


True

In [ ]:
my_set = {1, 2, 3, 4, 5}

my_set.remove(3)

print(my_set)  # Output: {1, 2, 4, 5}


{1, 2, 4, 5}


In [ ]:
conn = sqlite3.connect ('emisivos.db')
df.to_sql(name= 'emisivos', con=conn, if_exists= 'replace')

20602

In [ ]:
query = '''select e.IdLugarSalida
,b.`Lugar Salida`
, count(1) count_null
from emisivos e
,(select a.IdLugarSalida, a.`Lugar Salida` from  emisivos a where a.`Lugar Salida` is not null group by a.IdLugarSalida,a.`Lugar Salida`) b
where e.`Lugar Salida` is NULL
and b.IdLugarSalida = e.IdLugarSalida
group by e.IdLugarSalida,b.`Lugar Salida`'''

df_query = pd.read_sql_query(query, conn)

df_query

,IdLugarSalida,Lugar Salida,count_null
0,1,Colonia,340
1,2,Puerto de montevideo,76
2,3,Aeropuerto de Carrasco,484
3,4,Chuy,114
4,5,Carmelo,13
5,7,Fray Bentos,346
6,8,Paysandú,208
7,10,Salto,229
8,13,Rivera,66
9,14,Nueva Palmira,6


In [ ]:
def suma_nulos(column):
    null_count = column.isnull().sum()
    sin_datos_count = (column == "Sin Datos").sum()
    desc_sin_datos_count = (column == "Desconocido / Sin Datos").sum()
    s_d_count = (column == "S/D").sum()

    cant_nulos = null_count + sin_datos_count + desc_sin_datos_count + s_d_count
    return round(cant_nulos/len(column),4)

suma_nulos(df['Ocupacion'])

0.0112

In [ ]:
print(df.shape)

# Búsqueda de valores faltantes y valores únicos
display(df.nunique())
display((df == 0).sum(axis=0).to_frame().T.set_index(pd.Index(['zero_value_count'])).sum())
display((df == 'Sin Datos').sum(axis=0).to_frame().T.set_index(pd.Index(['SinDatos_value_count'])).sum())
df.isnull().sum().to_frame().T.set_index(pd.Index(['missing_value_count'])).sum()

(20602, 43)


IdLugarSalida                             15
Lugar Salida                              15
IdTranspSalidad                            6
Transporte Internacional de Salida         6
FechaSalida                             2920
IdFecSalida                             1621
FechaEntrada                            1418
IdFecEntrada                            1418
IdNacionalidad                            20
Pais                                      20
IdDeptoResidencia                         19
Departamento                              19
IdMotivo                                  10
Motivo                                    10
IdOcupacion                               21
Ocupacion                                 21
IdNivelEstudio                             8
Estudio                                    8
IdDestino                                 13
Destino                                   13
IdAlojamiento                             13
Alojamiento                               13
IdLugarIng

IdLugarSalida                              0
Lugar Salida                               0
IdTranspSalidad                            0
Transporte Internacional de Salida         0
FechaSalida                                0
IdFecSalida                                0
FechaEntrada                               0
IdFecEntrada                               0
IdNacionalidad                             0
Pais                                       0
IdDeptoResidencia                          0
Departamento                               0
IdMotivo                                   0
Motivo                                     0
IdOcupacion                               33
Ocupacion                                  0
IdNivelEstudio                            31
Estudio                                    0
IdDestino                                 20
Destino                                    0
IdAlojamiento                             19
Alojamiento                                0
IdLugarIng

IdLugarSalida                             0
Lugar Salida                              0
IdTranspSalidad                           0
Transporte Internacional de Salida        0
FechaSalida                               0
IdFecSalida                               0
FechaEntrada                              0
IdFecEntrada                              0
IdNacionalidad                            0
Pais                                      0
IdDeptoResidencia                         0
Departamento                              0
IdMotivo                                  0
Motivo                                    0
IdOcupacion                               0
Ocupacion                                33
IdNivelEstudio                            0
Estudio                                  31
IdDestino                                 0
Destino                                  20
IdAlojamiento                             0
Alojamiento                              19
IdLugarIngreso                  

IdLugarSalida                             0
Lugar Salida                           1993
IdTranspSalidad                           0
Transporte Internacional de Salida        0
FechaSalida                               0
IdFecSalida                               0
FechaEntrada                              0
IdFecEntrada                              0
IdNacionalidad                            0
Pais                                      0
IdDeptoResidencia                      2014
Departamento                           2014
IdMotivo                                610
Motivo                                  610
IdOcupacion                               0
Ocupacion                                 0
IdNivelEstudio                            0
Estudio                                   0
IdDestino                                 0
Destino                                   0
IdAlojamiento                             0
Alojamiento                               0
IdLugarIngreso                  

In [ ]:
display((df == 'Sin Datos').sum(axis=0).to_frame().T.set_index(pd.Index(['SinDatos_value_count'])).sum() +
        df.isnull().sum().to_frame().T.set_index(pd.Index(['missing_value_count'])).sum() +
        (df == 'Desconocido / Sin Datos').sum(axis=0).to_frame().T.set_index(pd.Index(['Desconocido_value_count'])).sum())

IdLugarSalida                             0
Lugar Salida                           1993
IdTranspSalidad                           0
Transporte Internacional de Salida        0
FechaSalida                               0
IdFecSalida                               0
FechaEntrada                              0
IdFecEntrada                              0
IdNacionalidad                            0
Pais                                      0
IdDeptoResidencia                      2014
Departamento                           2014
IdMotivo                                610
Motivo                                  610
IdOcupacion                               0
Ocupacion                               230
IdNivelEstudio                            0
Estudio                                  31
IdDestino                                 0
Destino                                  20
IdAlojamiento                             0
Alojamiento                              19
IdLugarIngreso                  

In [ ]:
print(df.info())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20602 entries, 0 to 20601
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   IdLugarSalida                        20602 non-null  int64  
 1   Lugar Salida                         18609 non-null  object 
 2   IdTranspSalidad                      20602 non-null  int64  
 3   Transporte Internacional de Salida   20602 non-null  object 
 4   FechaSalida                          20602 non-null  object 
 5   IdFecSalida                          20602 non-null  int64  
 6   FechaEntrada                         20602 non-null  object 
 7   IdFecEntrada                         20602 non-null  int64  
 8   IdNacionalidad                       20602 non-null  int64  
 9   Pais                                 20602 non-null  object 
 10  IdDeptoResidencia                    18588 non-null  float64
 11  Departamento                

,IdLugarSalida,IdTranspSalidad,IdFecSalida,IdFecEntrada,IdNacionalidad,IdDeptoResidencia,IdMotivo,IdOcupacion,IdNivelEstudio,IdDestino,...,GastoTransporteInternac,GatoTransporteLocal,GastoCultural,GastoTours,GastoCompras,GastoResto,Coef,CoefTot,latitud,longitud
count,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,18588.000000,19992.000000,20602.000000,20602.000000,20602.000000,...,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000,20602.000000
mean,5.823464,2.554218,14360.886176,14367.501456,1.419183,27.708737,2.035414,12.195709,4.668916,16.044025,...,368.732961,129.012120,87.451266,5.291424,257.934800,130.868796,212.533633,459.007001,-34.083811,-56.435726
std,4.212990,1.444270,711.410391,711.328919,3.468134,15.479640,1.766763,6.417587,1.128480,9.560664,...,985.352703,242.183745,187.157206,35.126745,501.851935,282.142647,174.477013,481.591600,1.200882,0.915610
min,1.000000,1.000000,13496.000000,13516.000000,1.000000,10.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.560000,2.560000,-34.971885,-58.427849
25%,3.000000,1.000000,13779.000000,13785.000000,1.000000,20.000000,1.000000,11.000000,4.000000,10.000000,...,0.000000,12.210000,0.000000,0.000000,0.000000,0.000000,109.770000,188.450000,-34.865083,-56.604992
50%,4.000000,2.000000,14172.000000,14178.000000,1.000000,21.000000,2.000000,13.000000,5.000000,10.000000,...,16.000000,62.260000,0.000000,0.000000,120.720000,33.970000,172.370000,318.300000,-34.789788,-56.185949
75%,8.000000,3.000000,14603.000000,14612.000000,1.000000,31.000000,2.000000,16.000000,6.000000,20.000000,...,243.810000,150.000000,104.000000,0.000000,255.575000,147.762500,258.630000,548.170000,-33.448853,-56.092397
max,18.000000,6.000000,15762.000000,15763.000000,70.000000,61.000000,10.000000,99.000000,7.000000,70.000000,...,16882.650000,8746.740000,4555.240000,1200.000000,17911.920000,8350.000000,3984.970000,7969.930000,-30.780011,-53.257832


In [ ]:
path = 'operadores.csv'
op = pd.read_csv(path, sep=';')
op.head()

,TipoOperador,Operador,Departamento,Localidad,Direccion,Telefono,Web,EMail,Longitud,Latitud
0,Inmobiliaria,DESTINO PUNTA INMOBILIARIA,MALDONADO,MALDONADO,SIMON BOLIVAR ENTRE THAILANDIA Y ALFEREZ APTO ...,095002383,www.destinopunta.com,info@destinopunta.com,-54,-34
1,Inmobiliaria,S/D,ROCHA,CHUY,1º DE AGOSTO 657,098574630,S/D,josegabrielbernasconi@gmail.com,-10000,-10000
2,Inmobiliaria,MONICA PUCURULL,MALDONADO,PUNTA BALLENA,RUTA INTERBALNEARIA KM 118 - PADRON 2055 MANZA...,094433282,www.monicapucurull.com.uy,monice@movinet.com.uy,-10000,-10000
3,Inmobiliaria,SPOT REAL ESTATE,MONTEVIDEO,MONTEVIDEO,POTOSI 1657/005A,26057127,www.spot.uy,carlos@spot.uy,-10000,-10000
4,Inmobiliaria,SANITAGO PEREYRA PROPIEDADES,MALDONADO,PUNTA DEL ESTE,AV FRANCISCO SALAZAR Y LENZINA - PARADA 2 - TO...,42485020,www.santiagopereyra.com.uy,info@santiagopereyra.com.uy,-54,-13


In [ ]:
conn = sqlite3.connect ('emisivos.db')
op.to_sql(name= 'operadores', con=conn, if_exists= 'replace')

3288

In [ ]:
path = 'calles.csv'
calles = pd.read_csv(path, sep=',')

conn = sqlite3.connect ('emisivos.db')
calles.to_sql(name= 'calles', con=conn, if_exists= 'replace')
calles.head(10)

,idcalle,calle,localidad,departamento
0,8042,BOULEVARD JOSE BATLLE Y ORDOÑEZ,MONTEVIDEO,MONTEVIDEO
1,8045,BRENDA,MONTEVIDEO,MONTEVIDEO
2,8050,CADIZ,MONTEVIDEO,MONTEVIDEO
3,8051,CAGANCHA,MONTEVIDEO,MONTEVIDEO
4,8052,CAMINO PASO DEL SAUCE,MONTEVIDEO,MONTEVIDEO
5,8052,CAMINO PASO DEL SAUCE,ABAYUBA,MONTEVIDEO
6,8053,CAMINO AL PASO HONDO,MONTEVIDEO,MONTEVIDEO
7,8056,CAMINO AL PUENTE ROTO,MONTEVIDEO,MONTEVIDEO
8,8059,CNO ALCION,MONTEVIDEO,MONTEVIDEO
9,8060,CAMINO ANAYA,SANTIAGO VAZQUEZ,MONTEVIDEO


In [ ]:
path = 'receptivos.json'
receptivos = pd.read_json(path)

conn = sqlite3.connect ('emisivos.db')
receptivos.to_sql(name= 'receptivos', con=conn, if_exists= 'replace')
receptivos.head(10)

,IdIngresos,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,FechaIngreso,IdFecIng,FechaEgreso,IdFecEgr,IdNacionalidad,Pais,...,GastoTotal,GastoAlojamiento,GastoAlimentacion,GastoTransporte,GastoCultural,GastoTours,GastoCompras,GastoOtros,Coef,CoefTot
0,3,Aeropuerto de Carrasco,1,Aereo,2017-02-22,13567,2017-03-03,13576,33,Ecuador,...,3213,1566,1134,0,0,0,0,513,129.35,388.04
1,18,Río Branco,2,Terrestre Auto,2017-01-05,13519,2017-01-09,13523,19,Brasil,...,2016,784,376,72,384,0,0,400,57.49,114.98
2,3,Aeropuerto de Carrasco,1,Aereo,2017-01-04,13518,2017-01-12,13526,30,Paraguay,...,3846,1568,752,160,0,0,566,800,63.94,127.89
3,7,Fray Bentos,2,Terrestre Auto,2017-01-18,13532,2017-01-26,13540,30,Paraguay,...,3696,2352,1128,216,0,0,0,0,53.25,159.76
4,7,Fray Bentos,2,Terrestre Auto,2017-01-02,13516,2017-01-17,13531,10,Argentina,...,6686,2940,1410,270,0,0,566,1500,142.84,285.68
5,2,Puerto de montevideo,5,Maritimo - Fluvial,2017-01-09,13523,2017-01-11,13525,59,Otro de Europa,...,289.78,157.98,104,15.8,0,0,0,12,141.33,282.65
6,3,Aeropuerto de Carrasco,1,Aereo,2017-01-20,13534,2017-01-27,13541,32,Colombia,...,1922,320,658,378,0,0,566,0,199.64,399.27
7,1,Colonia,5,Maritimo - Fluvial,2017-01-14,13528,2017-01-16,13530,19,Brasil,...,700,400,200,100,0,0,0,0,113.85,227.69
8,1,Colonia,5,Maritimo - Fluvial,2017-01-20,13534,2017-01-26,13540,1,Uruguay,...,2156,500,564,108,384,0,0,600,29.19,58.38
9,4,Chuy,2,Terrestre Auto,2017-01-26,13540,2017-01-30,13544,19,Brasil,...,1384,750,564,70,0,0,0,0,60.5,181.51


In [ ]:
c = receptivos.columns
display(receptivos[c[0:20]].head())
display(receptivos[c[20:40]].head())
receptivos[c[40:43]].head()

,IdIngresos,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,FechaIngreso,IdFecIng,FechaEgreso,IdFecEgr,IdNacionalidad,Pais,IdResidencia,Residencia,IdMotivo,Motivo,IdOcupacion,Ocupacion,IsEstudio,Estudio,IdDestinoLocalidad,Localidad
0,3,Aeropuerto de Carrasco,1,Aereo,2017-02-22,13567,2017-03-03,13576,33,Ecuador,50,Otras ciudades Sudamerica,99,Otros,20,"Deportista, Entrenador, Juez Dep",4,Secundaria completa,1,Montevideo
1,18,Río Branco,2,Terrestre Auto,2017-01-05,13519,2017-01-09,13523,19,Brasil,23,Santa Catarina,1,Ocio y vacaciones,12,"Director, gerente",6,Terciaria completa,3,Punta del Este
2,3,Aeropuerto de Carrasco,1,Aereo,2017-01-04,13518,2017-01-12,13526,30,Paraguay,30,Paraguay,1,Ocio y vacaciones,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,3,Punta del Este
3,7,Fray Bentos,2,Terrestre Auto,2017-01-18,13532,2017-01-26,13540,30,Paraguay,30,Paraguay,1,Ocio y vacaciones,11,"Patron, Com, Ind, Prod Agrop",4,Secundaria completa,3,Punta del Este
4,7,Fray Bentos,2,Terrestre Auto,2017-01-02,13516,2017-01-17,13531,10,Argentina,11,Buenos Aires,1,Ocio y vacaciones,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,3,Punta del Este


,IdDepartamentoDestino,Departamento,IdOtroDepartamento,Otro Departamento,IdOtraLocalidad,Otra Localidad,IdAlojamiento,Alojamiento,IdTranspLocal,TransporteLocal,IdEgresos,Lugar Egreso,IdTranspEgreso,Transporte Internacional de Egreso,IdDestino,Destino,Estadia,Gente,GastoTotal,GastoAlojamiento
0,1,Montevideo,0,Canelones,0,Parador Tajes,23,Hotel 3 estrellas,6,Otros,3,Aeropuerto de Carrasco,1,Aereo,3,Montevideo,9,3,3213,1566
1,2,Maldonado,1,Montevideo,0,Montevideo,24,Hotel 4 estrellas,1,Auto propio,18,Río Branco,2,Terrestre Auto,1,Punta del Este,4,2,2016,784
2,2,Maldonado,1,Montevideo,0,Montevideo,24,Hotel 4 estrellas,3,Taxi - Bus,3,Aeropuerto de Carrasco,1,Aereo,1,Punta del Este,8,2,3846,1568
3,2,Maldonado,1,Montevideo,0,Montevideo,23,Hotel 3 estrellas,1,Auto propio,7,Fray Bentos,2,Terrestre Auto,1,Punta del Este,8,3,3696,2352
4,2,Maldonado,1,Montevideo,0,Montevideo,23,Hotel 3 estrellas,1,Auto propio,7,Fray Bentos,2,Terrestre Auto,1,Punta del Este,15,2,6686,2940


,GastoAlimentacion,GastoTransporte,GastoCultural
0,1134,0,0
1,376,72,384
2,752,160,0
3,1128,216,0
4,1410,270,0


In [ ]:
import json

# Abrir el archivo JSON
with open('receptivos.json') as archivo_json:
    # Cargar el contenido del archivo en un diccionario
    dict_recep = json.load(archivo_json)



In [ ]:
dict_recep.index(dict_recep[-1])

47784

In [ ]:
len(dict_recep)

47785

In [ ]:
def val_tel (telefono):
    if telefono[0] == '0':
        return True if len(telefono) == 9 else False
    elif telefono[0] == '2' or telefono[0] == '4':
        return True if len(telefono) == 8 else False
    else:
        return False

val_tel('25252525')


True

In [ ]:
import smtplib
from email.mime.text import MIMEText

def validar_mail(email):
    sender = "your_email@gmail.com"  # Colocar e-mail de remitente
    password = "your_password"  # Colocar contraseña de remitente

    # Crear mail
    message = MIMEText("Mail de prueba")
    message["Subject"] = "Mail de prueba"
    message["From"] = sender
    message["To"] = email

    try:
        # Conectarse a SMTP server
        smtp_server = smtplib.SMTP("smtp.gmail.com", 587)
        smtp_server.starttls()

        # Iniciar sesión a mail
        smtp_server.login(sender, password)

        # Mandar mail
        smtp_server.sendmail(sender, email, message.as_string())

        # Cerrar la conexión
        smtp_server.quit()

        print("Mail enviado con éxito.")
        return True
    except Exception as e:
        print(f"Error en envío de mail {e}")
        return False

# Example usage
email = "recipient_email@example.com"  # Replace with the recipient's email address
send_test_email(email)


In [ ]:
import Levenshtein

def min_levenshtein_distance(string, referencial):
    # Se sacan los números de la columna de Dirección
    modified_string = ''.join(char for char in string if not char.isdigit())

    # Se calcula la mínima distancia de Levenshtein a algún valor del referencial
    # y se guarda el valor
    min_distance = float('inf')
    min_value = None

    for value in referencial:
        distance = Levenshtein.distance(modified_string, str(value))
        if distance < min_distance:
            min_distance = distance
            min_value = value

    return round(min_distance/(len(modified_string) + len(min_value)),4)

min_levenshtein_distance('PRESIDENTE FRANKLIN ROOSEVELT 260', calles['calle'])


0.2143

In [ ]:
calles[calles['calle'] == 'ITACUMBU']

,idcalle,calle,localidad,departamento
2737,12195,ITACUMBU,MONTEVIDEO,MONTEVIDEO
9918,21386,ITACUMBU,LAS PIEDRAS,CANELONES


In [ ]:
def duplicados_row (df,row):
    subset_columns = list(df.columns)
    cols_not_to_consider = []

    # Genero listado de columnas a partir de las cuales se buscan duplicados
    for col in cols_not_to_consider:
        subset_columns.pop(subset_columns.index(col))

    # Genero dataframe con duplicados
    df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)

    df_dup_row = df_duplicates[(df_duplicates[subset_columns] == df.loc[row][subset_columns]).all(axis=1)] # Dataframe con duplicados para cada registro duplicado

    if len(df_dup_row) == 0:
        no_duplicacion = 1
        no_contradiccion = 1
        return {'no_duplicacion': no_duplicacion, 'no_contradiccion': no_contradiccion}

    # Genero listas con los atributos que definiran si el duplicado es contradictorio o no

    check_list = [[]]

    to_remove = [None, 'Sin Datos', 'Desconocido / Sin Datos', 'S/D'] # valores a remover para chequear si los restantes son contradictorios

    # Chequeo si los duplicados son contradictorios
    for lista in check_list:
        contradiccion = []
        for value in to_remove:
            lista.remove(value) if value in lista else lista
        if len(lista) == len(set(lista)):
            contradiccion.append(True) # Existen valores distintos luego de sacar los nulos o faltantes
        else:
            contradiccion.append(False) # No existen valores distintos luego de sacar los nulos o faltantes

    if True in contradiccion:
         no_duplicacion = 1
         no_contradiccion = 0
    else:
        no_duplicacion = 0
        no_contradiccion = 1

    return {'no_duplicacion': no_duplicacion, 'no_contradiccion': no_contradiccion}


In [ ]:
for col in op.columns:
    print (f'Operadores.{col}')

Operadores.TipoOperador
Operadores.Operador
Operadores.Departamento
Operadores.Localidad
Operadores.Direccion
Operadores.Telefono
Operadores.Web
Operadores.EMail
Operadores.Longitud
Operadores.Latitud


In [ ]:
def duplicados(df):
    subset_columns = list(df.columns)
    cols_not_to_consider = []

    # Genero listado de columnas a partir de las cuales se buscan duplicados
    for col in cols_not_to_consider:
        subset_columns.pop(subset_columns.index(col))

    # Genero dataframe con duplicados
    df_duplicates = df[df.duplicated(subset=subset_columns,keep=False)].sort_values(by = subset_columns)

    dup_cont = [] # listado con índices de filas de duplicados contradictorios
    dup_no_cont = [] # listado con índices de filas de duplicados NO contradictorios

    for row in df_duplicates.index:
        df_dup_row = df_duplicates[(df_duplicates[subset_columns] == df.loc[row][subset_columns]).all(axis=1)] # Dataframe con duplicados para cada registro duplicado

        # Genero listas con los atributos que definiran si el duplicado es contradictorio o no
        #for col in cols_not_to_consider:
            #coef_list = [coef for coef in df_dup_row['Direccion']]
            #coef_tot_list = [coef_tot for coef_tot in df_dup_row['CoefTot']]
            #lat_list = [lat for lat in df_dup_row['latitud']]
            #lon_list = [lon for lon in df_dup_row['longitud']]
        check_list = [[]]

        to_remove = [None, 'Sin Datos', 'Desconocido / Sin Datos'] # valores a remover para chequear si los restantes son contradictorios

        # Chequeo si los duplicados son contradictorios
        for lista in check_list:
            contradiccion = []
            for value in to_remove:
                lista.remove(value) if value in lista else lista
            if len(lista) == len(set(lista)):
                contradiccion.append(True) # Existen valores distintos luego de sacar los nulos o faltantes
            else:
                contradiccion.append(False) # No existen valores distintos luego de sacar los nulos o faltantes

        dup_cont.append(row) if True in contradiccion else dup_no_cont.append(row)

    return {'ratio_cont' : round(len(dup_cont),4),'ratio_no_cont': round(len(dup_no_cont),4)}

duplicados(op)

{'ratio_cont': 616, 'ratio_no_cont': 0}

In [ ]:
(receptivos == '').sum()

IdIngresos                             45286
Lugar Ingreso                          45286
IdTranspIngreso                        45286
Transporte Internacional de Ingreso    45286
FechaIngreso                           45286
IdFecIng                               45286
FechaEgreso                            45286
IdFecEgr                               45286
IdNacionalidad                         45286
Pais                                   45286
IdResidencia                           45286
Residencia                             45286
IdMotivo                               45286
Motivo                                 45286
IdOcupacion                            45286
Ocupacion                              45286
IsEstudio                              18393
Estudio                                18393
IdDestinoLocalidad                     15490
Localidad                              15490
IdDepartamentoDestino                  15490
Departamento                           15490
IdOtroDepa

In [ ]:
c = receptivos.columns
display(receptivos[c[0:20]][174:180])
display(receptivos[c[20:40]][174:180])
receptivos[c[40:43]][174:180]

,IdIngresos,Lugar Ingreso,IdTranspIngreso,Transporte Internacional de Ingreso,FechaIngreso,IdFecIng,FechaEgreso,IdFecEgr,IdNacionalidad,Pais,IdResidencia,Residencia,IdMotivo,Motivo,IdOcupacion,Ocupacion,IsEstudio,Estudio,IdDestinoLocalidad,Localidad
174,1,Colonia,5,Maritimo - Fluvial,2017-02-01,13546,2017-02-13,13558,10,Argentina,14,Cordoba,1,Ocio y vacaciones,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,3,Punta del Este
175,3,Aeropuerto de Carrasco,1,Aereo,2017-02-17,13562,2017-02-27,13572,40,Chile,40,Chile,1,Ocio y vacaciones,2,Ama de casa,4,Secundaria completa,3,Punta del Este
176,1,Colonia,5,Maritimo - Fluvial,2017-02-01,13546,2017-02-15,13560,10,Argentina,11,Buenos Aires,1,Ocio y vacaciones,13,"Prof, Tecnico, Docente, Artista",6,Terciaria completa,3,Punta del Este
177,7,Fray Bentos,2,Terrestre Auto,2017-01-31,13545,2017-02-15,13560,10,Argentina,16,Pcia. Enre Rios,1,Ocio y vacaciones,11,"Patron, Com, Ind, Prod Agrop",4,Secundaria completa,3,Punta del Este
178,3,Aeropuerto de Carrasco,1,Aereo,2017-01-17,13531,2017-02-03,13548,32,Colombia,50,Otras ciudades Sudamerica,1,Ocio y vacaciones,11,"Patron, Com, Ind, Prod Agrop",7,Otros,3,Punta del Este
179,2,Puerto de montevideo,5,Maritimo - Fluvial,2017-02-06,13551,2017-02-20,13565,10,Argentina,11,Buenos Aires,1,Ocio y vacaciones,11,"Patron, Com, Ind, Prod Agrop",6,Terciaria completa,3,Punta del Este


,IdDepartamentoDestino,Departamento,IdOtroDepartamento,Otro Departamento,IdOtraLocalidad,Otra Localidad,IdAlojamiento,Alojamiento,IdTranspLocal,TransporteLocal,IdEgresos,Lugar Egreso,IdTranspEgreso,Transporte Internacional de Egreso,IdDestino,Destino,Estadia,Gente,GastoTotal,GastoAlojamiento
174,2,Maldonado,1,Montevideo,0,Montevideo,4,Vivienda arrendada,1,Auto propio,1,Colonia,5,Maritimo - Fluvial,1,Punta del Este,12,4,5432,2500
175,2,Maldonado,1,Montevideo,0,Montevideo,23,Hotel 3 estrellas,2,Auto alquilado,3,Aeropuerto de Carrasco,1,Aereo,1,Punta del Este,10,4,8832,3520
176,2,Maldonado,1,Montevideo,0,Montevideo,4,Vivienda arrendada,1,Auto propio,1,Colonia,5,Maritimo - Fluvial,1,Punta del Este,14,4,5920,2800
177,2,Maldonado,1,Montevideo,0,Montevideo,4,Vivienda arrendada,1,Auto propio,1,Colonia,5,Maritimo - Fluvial,1,Punta del Este,15,4,6272,2800
178,2,Maldonado,1,Montevideo,0,Montevideo,4,Vivienda arrendada,2,Auto alquilado,3,Aeropuerto de Carrasco,1,Aereo,1,Punta del Este,17,4,9544,2800
179,2,Maldonado,1,Montevideo,0,Montevideo,4,Vivienda arrendada,1,Auto propio,2,Puerto de montevideo,5,Maritimo - Fluvial,1,Punta del Este,14,5,7865,3000


,GastoAlimentacion,GastoTransporte,GastoCultural
174,1632,528,0
175,1840,1800,716
176,1904,616,0
177,2040,660,0
178,2312,3060,600
179,2380,770,750


In [ ]:
conn = sqlite3.connect ('emisivos.db')

query = '''select r.localidad, c.localidad
            from receptivos r
            inner JOIN calles c
            on upper(r.localidad) = c.localidad'''

df_loc = pd.read_sql_query(query, conn)